In [1]:
import pandas as pd
import seaborn as sns
import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import matthews_corrcoef, make_scorer
from sklearn.model_selection import cross_validate

import xgboost

from collections import Counter

sns.set_theme(style="whitegrid")
sns.set_palette(palette="Paired")
SEED: int = 42

# Load Data
We load the already processed data from the `data/processed` directory

In [2]:
train_data = pd.read_csv("../../data/processed/train_data_cleaned.csv", index_col="building_id")
y_train = pd.read_csv("../../data/processed/train_labels.csv", index_col="building_id", usecols=["building_id", "damage_grade"])

In [3]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train["damage_grade"].to_numpy())

In [4]:
print('Original counts of labels %s' % Counter(train_labels))

Original counts of labels Counter({1: 148259, 2: 87218, 0: 25124})


In [5]:
model = xgboost.XGBClassifier(n_estimators=100,
                              max_depth=20,
                              learning_rate=0.1,
                              subsample=0.8,
                              colsample_bytree=0.8,
                              random_state=42,
                              n_jobs=-1)

scoring = make_scorer(matthews_corrcoef)

In [6]:
%%time

cv_results = cross_validate(model, train_data, train_labels, cv=5,
                            scoring=scoring,
                            n_jobs=-1,
                            return_train_score=True)

CPU times: user 200 ms, sys: 167 ms, total: 366 ms
Wall time: 13min 25s


In [7]:
print(f"CV Test: {round(np.mean(cv_results['test_score']), 4)} +/- {round(np.std(cv_results['test_score']), 4)} MCC")

CV Test: 0.5125 +/- 0.0047 MCC


# Random Undersampling

In [8]:
from imblearn.under_sampling import RandomUnderSampler

In [9]:
rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(train_data, train_labels)
print('Resampled dataset shape %s' % Counter(y_rus))

Resampled dataset shape Counter({0: 25124, 1: 25124, 2: 25124})


In [10]:
%%time

cv_results_rus = cross_validate(model, X_rus, y_rus, cv=5,
                                scoring=scoring,  #"accuracy",
                                n_jobs=-1,
                                return_train_score=True,
                                verbose=1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  3.2min remaining:  4.8min


CPU times: user 150 ms, sys: 175 ms, total: 325 ms
Wall time: 3min 17s


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  3.3min finished


In [11]:
print(f"CV Test: {round(np.mean(cv_results_rus['test_score']), 4)} +/- {round(np.std(cv_results_rus['test_score']), 4)} MCC")

CV Test: 0.5765 +/- 0.0016 MCC


# Random Oversampling

In [12]:
from imblearn.over_sampling import RandomOverSampler

In [13]:
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(train_data, train_labels)
print('Resampled dataset shape %s' % Counter(y_ros))

Resampled dataset shape Counter({2: 148259, 1: 148259, 0: 148259})


In [14]:
%%time

cv_results_ros = cross_validate(model, X_ros, y_ros, cv=5,
                                scoring=scoring,
                                n_jobs=-1,
                                return_train_score=True, 
                                verbose=0)

CPU times: user 213 ms, sys: 97.8 ms, total: 311 ms
Wall time: 26min 12s


In [15]:
print(f"CV Test: {round(np.mean(cv_results_ros['test_score']), 4)} +/- {round(np.std(cv_results_ros['test_score']), 4)} MCC")

CV Test: 0.8056 +/- 0.029 MCC


# Combinations of over- and undersampling

## SMOTEENN

In [16]:
from imblearn.combine import SMOTEENN

In [17]:
%%time

sme = SMOTEENN(random_state=42)
X_sme, y_sme = sme.fit_resample(train_data, train_labels)

In [18]:
print('Resampled dataset shape %s' % Counter(y_sme))

Resampled dataset shape Counter({0: 125951, 2: 82828, 1: 50540})


In [19]:
%%time

cv_results_sme = cross_validate(model, X_sme, y_sme, cv=5,
                                scoring=scoring,
                                n_jobs=-1,
                                return_train_score=True, 
                                verbose=0)

CPU times: user 175 ms, sys: 156 ms, total: 331 ms
Wall time: 14min 26s


In [20]:
print(f"CV Test: {round(np.mean(cv_results_sme['test_score']), 4)} +/- {round(np.std(cv_results_sme['test_score']), 4)} MCC")

CV Test: 0.9066 +/- 0.0542 MCC


## SMOTETomek

In [21]:
from imblearn.combine import SMOTETomek

In [22]:
%%time

tom = SMOTETomek(random_state=42)
X_tom, y_tom = tom.fit_resample(train_data, train_labels)

CPU times: user 54min 3s, sys: 748 ms, total: 54min 4s
Wall time: 7min 34s


In [23]:
print('Resampled dataset shape %s' % Counter(y_sme))

Resampled dataset shape Counter({0: 125951, 2: 82828, 1: 50540})


In [24]:
%%time

cv_results_tom = cross_validate(model, X_tom, y_tom, cv=5,
                                scoring=scoring,
                                n_jobs=-1,
                                return_train_score=True, 
                                verbose=0)

CPU times: user 260 ms, sys: 236 ms, total: 496 ms
Wall time: 26min 28s


In [25]:
print(f"CV Test: {round(np.mean(cv_results_tom['test_score']), 4)} +/- {round(np.std(cv_results_tom['test_score']), 4)} MCC")

CV Test: 0.7485 +/- 0.1222 MCC


# Same methods but with train, test split

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef

X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

In [ ]:
model = xgboost.XGBClassifier(n_estimators=100,
                              max_depth=20,
                              learning_rate=0.1,
                              subsample=0.8,
                              colsample_bytree=0.8,
                              random_state=42,
                              n_jobs=-1)

model.fit(X_train, y_train)
predictions = model.predict(X_test)
score = matthews_corrcoef(y_test, predictions)

print(f"MCC: {round(score, 4)}")

In [28]:
%%time

# Undersampling
rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_rus))

model = xgboost.XGBClassifier(n_estimators=100,
                              max_depth=20,
                              learning_rate=0.1,
                              subsample=0.8,
                              colsample_bytree=0.8,
                              random_state=42,
                              n_jobs=-1)

model.fit(X_rus, y_rus)
predictions = model.predict(X_test)
score = matthews_corrcoef(y_test, predictions)

print(f"MCC: {round(score, 4)}")

Resampled dataset shape Counter({0: 19954, 1: 19954, 2: 19954})
MCC: 0.4546
CPU times: user 4min 59s, sys: 6.75 s, total: 5min 6s
Wall time: 44.8 s


In [29]:
%%time 

# Oversampling
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_ros))

model = xgboost.XGBClassifier(n_estimators=100,
                              max_depth=20,
                              learning_rate=0.1,
                              subsample=0.8,
                              colsample_bytree=0.8,
                              random_state=42,
                              n_jobs=-1)

model.fit(X_ros, y_ros)
predictions = model.predict(X_test)
score = matthews_corrcoef(y_test, predictions)

print(f"MCC: {round(score, 4)}")

Resampled dataset shape Counter({1: 118772, 2: 118772, 0: 118772})
MCC: 0.5069
CPU times: user 26min 16s, sys: 19.9 s, total: 26min 36s
Wall time: 4min 30s


In [30]:
%%time 

# SMOTEENN
sme = SMOTEENN(random_state=42)
X_sme, y_sme = sme.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_sme))

model = xgboost.XGBClassifier(n_estimators=100,
                              max_depth=20,
                              learning_rate=0.1,
                              subsample=0.8,
                              colsample_bytree=0.8,
                              random_state=42,
                              n_jobs=-1)

model.fit(X_sme, y_sme)
predictions = model.predict(X_test)
score = matthews_corrcoef(y_test, predictions)

print(f"MCC: {round(score, 4)}")

Resampled dataset shape Counter({0: 101391, 2: 65787, 1: 39414})
MCC: 0.4911
CPU times: user 54min 18s, sys: 12.3 s, total: 54min 30s
Wall time: 8min 31s


In [31]:
%%time

# SMOTETomek
tom = SMOTETomek(random_state=42)
X_tom, y_tom = tom.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_tom))

model = xgboost.XGBClassifier(n_estimators=100,
                              max_depth=20,
                              learning_rate=0.1,
                              subsample=0.8,
                              colsample_bytree=0.8,
                              random_state=42,
                              n_jobs=-1)

model.fit(X_tom, y_tom)
predictions = model.predict(X_test)
score = matthews_corrcoef(y_test, predictions)

print(f"MCC: {round(score, 4)}")

Resampled dataset shape Counter({0: 117355, 2: 111448, 1: 110475})
MCC: 0.5121
CPU times: user 1h 6min 16s, sys: 20.4 s, total: 1h 6min 36s
Wall time: 10min 30s


# Conclusion

Positive results: 
- Random undersampling increased the MCC of about 0.06